In [4]:
from yolov3 import cfg
from yolov3.utils import *
from yolov3.visualize import *

model=cfg.MODEL

img_path='C:/Users/phiho/Projects/GAN_Yolov5/Dataset/train_dog_cat_person/299x299/dog/000000002754_3.jpg'
class_names=cfg.CLASS_NAMES
device=cfg.DEVICE
save_to='yolov3/preds/dog.jpg'
model.eval()

import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import cv2
import torchvision

class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None, target_size=(608, 608), device='cpu'):
        self.dataset = torchvision.datasets.ImageFolder(root_dir, transform=transform)
        self.target_size = target_size
        self.device = device
        self.img_paths = [path for path, _ in self.dataset.samples]

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        img, label = self.dataset[idx]
        img = cv2.cvtColor(img.cpu().numpy().transpose((1, 2, 0)), cv2.COLOR_RGB2BGR)
        img = cv2.resize(img, self.target_size)
        img = torch.from_numpy(img.transpose(2, 0, 1)).float().unsqueeze(0)
        img = img.to(self.device)

        return img, label, img_path
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

COCO_ROOT = "C:/Users/phiho/Projects/Dataset/MS-COCO/COCO_81/coco81/"
# Tạo dataset
coco_dataset = CustomDataset(root_dir=COCO_ROOT, 
                             transform=transforms.ToTensor(), 
                             target_size=(608, 608), 
                             device=device)
dataloader = DataLoader(coco_dataset, batch_size=32, shuffle=True, drop_last=True)

In [9]:
from tqdm import tqdm
for i, data in tqdm(enumerate(dataloader, start=0)):
    images,labels,p = data
    break

0it [00:00, ?it/s]


In [17]:
nms_thresh = 0.5
conf_thres = 0.0005
batch = images.shape[0]
res = torch.zeros(batch, 80).to(device)
images = images.to(device)
for j, im in enumerate(images):
    print(im.shape)
    im = cv2.cvtColor(im.cpu().detach().numpy().transpose((1, 2, 0)), cv2.COLOR_RGB2BGR)
    im = cv2.resize(im, (608, 608))
    im = torch.from_numpy(im.transpose(2, 0, 1)).float().unsqueeze(0)
    im = im.to(device)
    output = model(im)
    boxes = []
    for i in range(3):
        boxes += find_all_boxes_2(
            output[i].data,
            device,
            conf_thres,
            model.num_classes,
            model.anchors[i],
            model.num_anchors)[0]
    boxes = nms_combined(boxes, nms_thresh)
    res = torch.zeros((80), device=device)
    conf = boxes[:, -2]
    cls = boxes[:, -1].long()
    for c in range(80):
        mask = (cls == c)
        if mask.any():
            res[j][c] = conf[mask].max()

torch.Size([1, 3, 608, 608])


ValueError: axes don't match array

In [71]:
j = 213332
conf_thresh=0.001
nms_thresh=0.5
# img = cv2.cvtColor(coco_dataset[j][0].cpu().numpy().transpose((1, 2, 0)), cv2.COLOR_RGB2BGR)
# img = cv2.resize(img, (608, 608))
# img = torch.from_numpy(img.transpose(2, 0, 1)).float().unsqueeze(0)
# img = img.to(device)
img = coco_dataset[j][0]
output = model(img)

boxes = []
for i in range(3):
    boxes += find_all_boxes_2(
        output[i].data,
        device,
        conf_thresh,
        model.num_classes,
        model.anchors[i],
        model.num_anchors)[0]

boxes = nms_combined(boxes, nms_thresh)

# pred_img = plot_boxes_cv2(cv2.imread(img_path), [], save_to, class_names)
# pred_img = plot_boxes_cv2(cv2.cvtColor(coco_dataset[j][0].cpu().numpy().transpose((1, 2, 0)), cv2.COLOR_RGB2BGR)*255, boxes, save_to, class_names)
# pred_img = plot_boxes_cv2(coco_dataset[j][0].squeeze(0).cpu().numpy().transpose((1, 2, 0), boxes, save_to, class_names)
# pred_img
# print(coco_dataset[j][2])
# print(boxes)
# for *_, conf, cls in boxes:
#     print(f"{class_names[int(cls.item())]} : {conf.item()}")
res = torch.zeros((80), device=device)
conf = boxes[:, -2]
cls = boxes[:, -1].long()
res.scatter_add_(0, cls, conf)
print(torch.argmax(res).item())

res = torch.zeros((80), device=device)
conf = boxes[:, -2]
cls = boxes[:, -1].long()
for c in range(80):
    mask = (cls == c)
    if mask.any():
        res[c] = conf[mask].max()
print(torch.argmax(res).item())

res = torch.zeros((80), device=device)
for *_, conf, cls in boxes:
    c = int(cls.item())
    if res[c] < conf:
        res[c] = conf
print('Standard', torch.argmax(res).item())

62
8
Standard 8


In [41]:
j

213

4


In [1]:
import torch

random_number = torch.randint(0, 80, (1,)).item()  # Tạo một số nguyên ngẫu nhiên từ 0 đến 79
print(random_number)


39


In [ ]:
predict(
    model=cfg.MODEL,
    conf_thresh=0.7,
    nms_thresh=0.5,
    img_path='C:/Users/phiho/Projects/GAN_Yolov5/Dataset/train_dog_cat_person/299x299/dog/000000002754_3.jpg',
    class_names=cfg.CLASS_NAMES,
    device=cfg.DEVICE,
    save_to='yolov3/preds/dog.jpg'
    )